# 01 - Data Exploration and labelling

In Maffei.xlsx, in the sheet summary statistics, it is reported which phage causes lysis in which E.coli strain. Each row represents a phage, and the columns represent the host. The classification is binary (1=lysis, 0=no-lysis). For the sake of the project, we will consider successful infection only when there is lysis.

## Import libraries

In [6]:
import pandas as pd
import numpy as np
import os

# Import user defined libraries
import sys
sys.path.append("../src")
from fastaLoad import fasta

## 1.Load data


### 1.1 Maffei
Let's load data from Maffei. The excel sheet is not clean for import and some rows and columns need to be skipped, as well as some header renamed.\
Also, we only care about the phage identifiers and lysis data, so we'll only keep those columns.\
We obtain a maffei dataset with the first 4 columns identifying the virus: bas, family, genus and phage.\
The following columns state host strains and contain binary values for lysis or not: 1 = lysis

In [7]:
# Maffei raw data
RAW_PATH = os.path.join('..', 'data', 'raw')

maffei = pd.read_excel(os.path.join(RAW_PATH, 'Maffei.xlsx'), sheet_name='summary statistics', skiprows=1, header=[0,1,2])
maffei.drop(columns=maffei.columns[0], inplace=True)

# Store the first level of the first 4 headers in a list
header = [col[0] for col in maffei.columns.tolist()[:4]]
header.append('lysis observed (yes=1, no=0)')

# Keep only selected headers
maffei = maffei.loc[:, header]

# Drop odd columns up to 7
maffei.drop(columns=maffei.columns[1:9:2], inplace=True)

# Remove the first two rows of headers
maffei.columns = maffei.columns.droplevel([0,1])

# Substitute the first 4 headers
maffei = maffei.set_axis(header[:4] + maffei.columns[4:].tolist(), axis=1)

# Drop last column (who was kept for some mysterious reason)
maffei = maffei.iloc[:, :10]

# Rename columns
change_names = dict()
change_names= {'ICTV family (subfamily)': 'family',
               'ICTV genus': 'genus',
               'Bas#': 'bas',
               'Phage (name)': 'phage'}
maffei.rename(columns=change_names, inplace=True)
del change_names
maffei.columns = maffei.columns.str.replace(' ', '_')
maffei.columns = maffei.columns.str.replace('.', '')

# Move bas column to first position
bas = maffei.pop('bas')
maffei.insert(0, 'bas', bas)
del bas

# Define schema
header = maffei.columns[:4].tolist()
maffei[header] = maffei[header].astype('string')
maffei.iloc[:, 4:] = maffei.iloc[:, 4:].astype('int64') # Could be bool or categorical, but int64 is more convenient for machine learning purposes
del header

display(maffei)
maffei.dtypes


,bas,family,genus,phage,E_coli_UTI89,E_coli_CFT073,E_coli_55989,S_e_Typhimurium_12023s,S_e_Typhimurium_SL1344,E_coli_B_REL606
0,Bas01,Drexlerviridae; Braunvirinae,Guelphvirus,Escherichia phage AugustePiccard,0,0,0,0,0,1
1,Bas02,Drexlerviridae; Braunvirinae,Rtpvirus,Escherichia phage JeanPiccard,0,0,0,0,0,1
2,Bas03,Drexlerviridae; Braunvirinae,Rtpvirus,Escherichia phage JulesPiccard,0,0,0,0,0,1
3,Bas04,Drexlerviridae; Tempevirinae,Warwickvirus,Escherichia phage FritzSarasin,0,1,0,0,0,1
4,Bas05,Drexlerviridae; Tempevirinae,Warwickvirus,Escherichia phage PeterMerian,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
73,phage N4,Schitoviridae; Enquatrovirinae,Enquatrovirus,n.a.,1,0,1,0,0,0
74,Bas69,Schitoviridae; Enquatrovirinae,Enquatrovirus,Escherichia phage AlfredRasser,1,0,1,0,0,0
75,lambdavir,Siphoviridae,Lambdavirus,n.a.,0,0,0,0,0,1
76,P1vir,Myoviridae,Punavirus,n.a.,0,0,0,0,0,1


bas                       string[python]
family                    string[python]
genus                     string[python]
phage                     string[python]
E_coli_UTI89                       int64
E_coli_CFT073                      int64
E_coli_55989                       int64
S_e_Typhimurium_12023s             int64
S_e_Typhimurium_SL1344             int64
E_coli_B_REL606                    int64
dtype: object

### 1.2 Basel receptors
Basel receptors describes the receptors for each phage in Maffei's paper

In [8]:
receptors = pd.read_csv(os.path.join(RAW_PATH, 'basel_receptors.csv'), sep=';')

# Rename columns
change_names = {'ICTV genus': 'genus',
                'Bas##': 'bas',
                'Phage (name)': 'phage',
                'closest relative (BLASTN total score)': 'closest relative'} 
receptors.rename(columns=change_names, inplace=True)
del change_names
receptors.columns = receptors.columns.str.replace(' ', '_')

# Move genus and phage columns to second and third position
genus = receptors.pop('genus')
phage = receptors.pop('phage')
receptors.insert(1, 'genus', genus)
receptors.insert(2, 'phage', phage)
del genus, phage

# Define schema
receptors = receptors.astype('string')

display(receptors)
receptors.dtypes

,bas,genus,phage,morphotype,closest_relative,primary_receptor,terminal_receptor,receptor
0,Bas01,Rtpvirus,Escherichia phage AugustePiccard,siphovirus,RTP (AM156909.1),LPS / O-antigen?,LptD,n.a.
1,Bas02,Guelphvirus,Escherichia phage JeanPiccard,siphovirus,CEB_EC3a (NC_047812.1),LPS / O-antigen?,LptD,n.a.
2,Bas03,Guelphvirus,Escherichia phage JulesPiccard,siphovirus,CEB_EC3a (NC_047812.1),LPS / O-antigen?,FhuA,n.a.
3,Bas04,Warwickvirus,Escherichia phage FritzSarasin,siphovirus,tonnikala (NC_049817.1),LPS / O-antigen?,BtuB,n.a.
4,Bas05,Warwickvirus,Escherichia phage PeterMerian,siphovirus,XY3 (MN781674.1),LPS / O-antigen?,FhuA,n.a.
...,...,...,...,...,...,...,...,...
73,n.a.,Teseptimavirus,Escherichia phage T7 (reference),podovirus,n.a.,n.a.,n.a.,rough LPS
74,n.a.,Enquatrovirus,Escherichia phage N4 (reference),podovirus,n.a.,ECA,NfrA?,n.a.
75,n.a.,Lambdavirus,Escherichia phage lambdavir (reference),siphovirus,n.a.,none,LamB,n.a.
76,n.a.,Punavirus,Escherichia phage P1vir (reference),myovirus,n.a.,n.a.,n.a.,rough LPS


bas                  string[python]
genus                string[python]
phage                string[python]
morphotype           string[python]
closest_relative     string[python]
primary_receptor     string[python]
terminal_receptor    string[python]
receptor             string[python]
dtype: object

### 1.3 Phage protein labels
Phage protein is a list of preotein sequences IDs (FASTA) for the viruses in Maffei's paper. It states the categorization of these proteins according to the following Neural Networks:

        PhaNNS
        PhageRBPdetect
        ESM-based method developed by Yumeng\
\
Interpreting the Labels:

        Columns 1-11 present the label scores for PhaNNs, with values approaching 10 signifying high confidence.
        Column 12 provides PhaNNs' confidence level.
        Column 13 indicates PhageRBPdetect predictions: 1 denotes an RBP prediction, while 0 signifies otherwise.
        Column 14 offers PhageRBPdetect scores, with values close to 1 signifying strong confidence.
        Column 15 presents the ESM-based label.
        Column 16 features 1 if the label in Column 15 is "tail_fiber."

In [9]:
proteins = pd.read_csv(os.path.join(RAW_PATH, 'phage_proteins_labels.csv'), sep=',')

# Rename columns
proteins.columns = proteins.columns.str.replace('.', '_')
proteins.rename(columns=lambda x: x.lower() if proteins.columns.get_loc(x) < 10 else x, inplace=True)
proteins.rename(columns=lambda x: x.lower() if proteins.columns.get_loc(x) in [11, 12] else x, inplace=True)
proteins.rename(columns={'name': 'seqID'}, inplace=True)

# Define schema
proteins['seqID'] = proteins['seqID'].astype('string')
proteins.iloc[:, 1:13] = proteins.iloc[:, 1:13].astype('float64')
proteins['PhageRBPdetect_prediction'] = proteins['PhageRBPdetect_prediction'].astype('int64')
proteins['PhageRBDdetect_score'] = proteins['PhageRBDdetect_score'].astype('float64')
proteins['ESM_based_label'] = proteins['ESM_based_label'].astype('string')
proteins['ESM_based_fiber_prediction'] = proteins['ESM_based_fiber_prediction'].astype('int64')

display(proteins)
# proteins.dtypes

,seqID,major_capsid,minor_capsid,baseplate,major_tail,minor_tail,portal,tail_fiber,tail_shaft,collar,HTJ,other,confidence,PhageRBPdetect_prediction,PhageRBDdetect_score,ESM_based_label,ESM_based_fiber_prediction
0,lcl|MZ501051.1_prot_QXV76132.1_1,0.22,0.84,0.18,0.21,0.34,0.85,0.49,0.24,0.46,0.38,5.79,0.98,0,2.403247e-05,other,0
1,lcl|MZ501051.1_prot_QXV76133.1_2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00,1.00,0,3.570606e-08,other,0
2,lcl|MZ501051.1_prot_QXV76134.1_3,0.35,1.01,0.71,0.26,1.30,0.97,0.68,0.25,0.24,0.73,3.51,0.97,0,1.817806e-06,other,0
3,lcl|MZ501051.1_prot_QXV76135.1_4,0.01,0.23,0.02,0.01,0.03,0.10,0.09,0.01,0.16,0.03,9.32,1.00,0,1.184212e-05,other,0
4,lcl|MZ501051.1_prot_QXV76136.1_5,0.05,0.07,0.06,0.05,0.13,0.09,0.03,0.07,0.08,0.07,9.28,1.00,0,8.151848e-05,other,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35411,lcl|HQ259105.1_prot_AEL79679.1_75,0.41,1.24,0.25,0.09,0.18,0.86,0.43,1.48,0.21,0.63,4.22,0.97,0,8.595828e-07,other,0
35412,lcl|HQ259105.1_prot_AEL79680.1_76,0.02,0.17,0.05,0.11,0.12,0.14,0.09,0.04,0.10,5.88,3.28,0.80,0,5.343628e-08,other,0
35413,lcl|HQ259105.1_prot_AEL79681.1_77,0.01,0.07,0.07,0.00,0.05,0.01,0.06,0.00,0.01,0.46,9.25,1.00,0,2.539302e-05,other,0
35414,lcl|HQ259105.1_prot_AEL79682.1_78,0.00,0.00,0.00,0.00,0.01,0.01,0.02,0.00,0.02,0.00,9.94,1.00,0,4.384229e-07,other,0


### 1.4 Basel proteome
basel_proteome.fasta includes the full proteome of each phage mentioned in the Maffei et al. paper. This covers a total of 248 phages that have been sequenced and annotated. Specifically, this includes 69 phages from the Basel collection.

Let's load the fasta file in a dataframe, with each tag of the header as a column, as well as the sequence

In [15]:
# basel_proteome = fasta(os.path.join(RAW_PATH, 'basel_proteome.fasta'), 'phage')
# basel_proteome.df['bas'] = basel_proteome.df['locus_tag'].str.split('_', expand=True)[0]
# display(basel_proteome.df.loc[basel_proteome.df['protein'] == 'tail fiber', :])
display(basel_proteome.df)
basel_proteome.df.dtypes
basel_proteome.df.shape

,seqID,locus_tag,protein,protein_id,location,gbkey,sequence,gene,db_xref,frame,partial,exception,bas
0,lcl|MZ501051.1_prot_QXV76132.1_1,bas01_0001,terminase small subunit,QXV76132.1,1..507,CDS,MSKAALKMGEGNFKALYNKKYGDIAMVAINRKYTPEEVFDFAVRYF...,NaN,NaN,NaN,NaN,NaN,bas01
1,lcl|MZ501051.1_prot_QXV76133.1_2,bas01_0002,hypothetical protein,QXV76133.1,526..621,CDS,MKGFIKLFIWYYLLTSISLCVFMLVVKLWLI,NaN,NaN,NaN,NaN,NaN,bas01
2,lcl|MZ501051.1_prot_QXV76134.1_3,bas01_0003,terminase large subunit,QXV76134.1,609..2177,CDS,MANLIWEEMTSQEKLAVKAISEHSFEGFLRCWFSITQGERYIPNWH...,NaN,NaN,NaN,NaN,NaN,bas01
3,lcl|MZ501051.1_prot_QXV76135.1_4,bas01_0004,putative homing endonuclease,QXV76135.1,2321..2716,CDS,MVAGSLSGNGYLHIRIGDRRVKNHLIIWEMHNGRIPEGMEIDHINH...,NaN,NaN,NaN,NaN,NaN,bas01
4,lcl|MZ501051.1_prot_QXV76136.1_5,bas01_0005,hypothetical protein,QXV76136.1,2788..3168,CDS,MTKKSKAVYLGNTEGEYYGFTVGNEYDVHNYETEDNFIGTFGDDGG...,NaN,NaN,NaN,NaN,NaN,bas01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35411,lcl|HQ259105.1_prot_AEL79679.1_75,gp69,hypothetical protein,AEL79679.1,complement(69352..70041),CDS,MRELTKQQIINALPANFKNSVSQELVDTINNITQDQLVAEAFRENF...,NaN,NaN,NaN,NaN,NaN,gp69
35412,lcl|HQ259105.1_prot_AEL79680.1_76,gp70,hypothetical protein,AEL79680.1,70225..70503,CDS,MRVIDSKDEYSEIDHTKAGFMAGDIILTHDEDTFHELSGMMLLENR...,NaN,NaN,NaN,NaN,NaN,gp70
35413,lcl|HQ259105.1_prot_AEL79681.1_77,gp71,hypothetical protein,AEL79681.1,70705..71103,CDS,MSIYAFDIDITGVLRKQEPYYLGEPEVVSKFTAPSQSVFLPDCEES...,NaN,NaN,NaN,NaN,NaN,gp71
35414,lcl|HQ259105.1_prot_AEL79682.1_78,gp71.1,hypothetical protein,AEL79682.1,71069..71383,CDS,MKFIMTIPSFVEGESMYTTDFSLEEVMNSNPQQVILVPTLENYHEV...,NaN,NaN,NaN,NaN,NaN,gp71.1


(35416, 13)

### 1.5 K12 proteome

In [11]:
k12_proteome = fasta(os.path.join(RAW_PATH, 'K12.fasta'), 'k12')
display(k12_proteome.df.head())

,id,name,OS,OX,GN,PE,SV,sequence
0,sp|A5A616|MGTS_ECOLI,Small protein MgtS,Escherichia coli (strain K12),83333,mgtS,1,1,MLGNMNVFMAVLGIILFSGFLAAYFSHKWDD
1,sp|O32583|THIS_ECOLI,Sulfur carrier protein ThiS,Escherichia coli (strain K12),83333,thiS,1,1,MQILFNDQAMQCAAGQTVHELLEQLDQRQAGAALAINQQIVPREQW...
2,sp|P00350|6PGD_ECOLI,"6-phosphogluconate dehydrogenase, decarboxylat...",Escherichia coli (strain K12),83333,gnd,1,2,MSKQQIGVVGMAVMGRNLALNIESRGYTVSIFNRSREKTEEVIAEN...
3,sp|P00363|FRDA_ECOLI,Fumarate reductase flavoprotein subunit,Escherichia coli (strain K12),83333,frdA,1,3,MQTFQADLAIVGAGGAGLRAAIAAAQANPNAKIALISKVYPMRSHT...
4,sp|P00370|DHE4_ECOLI,NADP-specific glutamate dehydrogenase,Escherichia coli (strain K12),83333,gdhA,1,1,MDQTYSLESFLNHVQKRDPNQTEFAQAVREVMTTLWPFLEQNPKYR...


In [12]:
display(k12_proteome.df.loc[k12_proteome.df['GN'].str.lower() == 'nfra'.lower()])

,id,name,OS,OX,GN,PE,SV,sequence
2584,sp|P31600|NFRA_ECOLI,Bacteriophage adsorption protein A,Escherichia coli (strain K12),83333,nfrA,1,1,MKENNLNRVIGWSGLLLTSLLSTSALADNIGTSAEELGLSDYRHFV...


### 1.6 Save clean datesets in <code>data/interim/clean<code>

In [13]:
CLEAN_PATH = os.path.join('..', 'data', 'interim', 'clean')

if not os.path.exists(CLEAN_PATH):
    os.makedirs(CLEAN_PATH)

maffei.to_csv(os.path.join(CLEAN_PATH, '0_maffei.csv'), index=False)
receptors.to_csv(os.path.join(CLEAN_PATH, '0_receptors.csv'), index=False)
proteins.to_csv(os.path.join(CLEAN_PATH, '0_proteins.csv'), index=False)
basel_proteome.df.to_csv(os.path.join(CLEAN_PATH, '0_basel_proteome.csv'), index=False)
k12_proteome.df.to_csv(os.path.join(CLEAN_PATH, '0_k12_proteome.csv'), index=False)


## 2 Data exploration

Some of basel receptors are terminal (aka secondary) and some are primary. In both columns there are both LPS and proteins. We only care about proteins, so it might be a good idea to add a column 'receptor_protein' which selects the proteic one from the two. Also, the 'receptor' column has no proteins, so it might be useless for further analysis. \

Cleaning to be considered:

    1. Some receptor entries have a question mark. For the moment, I will discard them
    2. If both primary and terminal receptors are n.a., I will discard the entry

In [20]:
# Get a set of the elements in a df
GN_set = set(k12_proteome.df['GN'].str.lower().tolist())

# Create a subset that contains only the rows where either primary_receptor or terminal_receptor are in GN_set
prot_receptors = receptors[(receptors['primary_receptor'].str.lower().isin(GN_set)) | (receptors['terminal_receptor'].str.lower().isin(GN_set))]
prot_receptors['receptor_protein'] = np.where(prot_receptors['primary_receptor'].str.lower().isin(GN_set), prot_receptors['primary_receptor'], 
                                      np.where(prot_receptors['terminal_receptor'].str.lower().isin(GN_set), prot_receptors['terminal_receptor'], np.nan))
prot_receptors.drop(columns=['primary_receptor', 'terminal_receptor', 'receptor'], inplace=True)

display(prot_receptors)

C:\Users\pf259\AppData\Local\Temp\ipykernel_2812\1070805616.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prot_receptors['receptor_protein'] = np.where(prot_receptors['primary_receptor'].str.lower().isin(GN_set), prot_receptors['primary_receptor'],
C:\Users\pf259\AppData\Local\Temp\ipykernel_2812\1070805616.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prot_receptors.drop(columns=['primary_receptor', 'terminal_receptor', 'receptor'], inplace=True)


,bas,genus,phage,morphotype,closest_relative,receptor_protein
0,Bas01,Rtpvirus,Escherichia phage AugustePiccard,siphovirus,RTP (AM156909.1),LptD
1,Bas02,Guelphvirus,Escherichia phage JeanPiccard,siphovirus,CEB_EC3a (NC_047812.1),LptD
2,Bas03,Guelphvirus,Escherichia phage JulesPiccard,siphovirus,CEB_EC3a (NC_047812.1),FhuA
3,Bas04,Warwickvirus,Escherichia phage FritzSarasin,siphovirus,tonnikala (NC_049817.1),BtuB
4,Bas05,Warwickvirus,Escherichia phage PeterMerian,siphovirus,XY3 (MN781674.1),FhuA
5,Bas06,Hanrivervirus,Escherichia phage KarlJaspers,siphovirus,herni (NC_049823.1),BtuB
6,Bas07,Hanrivervirus,Escherichia phage JakobBernoulli,siphovirus,PGN590 (NC_049830.1),FhuA
7,Bas08,Tlsvirus,Escherichia phage DanielBernoulli,siphovirus,TLS (AY308796.1),TolC
8,Bas09,Changchunvirus,Escherichia phage PaulSarasin,siphovirus,W011D (MK778457.1),FhuA
11,Bas12,Sertoctavirus,Escherichia phage BrunoManser,siphovirus,SRT8 (NC_042043.1),FhuA


Next, we need to select from each Bas (from receptors), which of its proteins (from basel_proteome) is expected to interact with that kind of receptor. It might be a good idea to define a dictionary to do this. 

After selecting the involved protein(s) for each Bas, we'll have to link those proteins with the corresponding protein in the k12_proteome. This pairs will be labeled as interacting. All the others as non-interacting